# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-7f8b8f0ef0-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Tv series HIMYM explorative search") 


Consider the following exploratory scenario:


> we are interested in the TV series "How I met your mother" and we want to investigate the main aspects related to the actors and directors involved in the production, know the numerber of seasons and check what are the episodes which got the higher success/impact.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P4969`    | derivative work      | predicate |
| `wd:Q147235` | How I met your mother        | node |
| `wd:Q23831` | The Office (US)        | node |



Also consider

```
wd:Q23831 ?p ?obj .
```

is the BGP to retrieve all **properties of The Office (US)**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should

1. Identify the BGP for tv series

2. Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

3. Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

4. Check who is the actor who acted in more films while working on "How I met your mother" (the result set must be a list of triples actor/actress IRI, label and #films).

5. Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

6. Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

7. Consider the actors who are members of the cast of HIMYM. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

## Task 1
Identify the BGP for tv series

In [3]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q23831 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/

In [4]:
#P136 genre
queryString = """
SELECT DISTINCT ?sName ?oName
WHERE {
   ?s wdt:P136 ?o .
   ?s sc:name ?sName.
   ?o sc:name ?oName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('sName', 'Harpist of Keros'), ('oName', 'Cycladic art')]
[('sName', 'Ayinde Barrister'), ('oName', 'fuji music')]
[('sName', 'Crying for No Reason'), ('oName', 'breakstep')]
[('sName', 'Chapliniana'), ('oName', 'ballet film')]
[('sName', 'Scribblenauts Unmasked: A DC Comics Adventure'), ('oName', 'emergent gameplay')]
[('sName', 'Ajin'), ('oName', 'supernatural anime and manga')]
[('sName', 'Rygar'), ('oName', 'spacesynth')]
[('sName', 'Amoeba Cleansing Syndrome'), ('oName', 'progressive soul')]
[('sName', 'Love Steaks'), ('oName', 'Mumblecore')]
[('sName', 'Nimrod Borenstein'), ('oName', 'ballet')]
[('sName', 'Shakey Jake Harris'), ('oName', 'Chicago blues')]
[('sName', 'Da Funk'), ('oName', 'acid house')]
[('sName', 'Lee Kwang-ki'), ('oName', 'trot')]
[('sName', 'Seo In-ah'), ('oName', 'trot')]
[('sName', 'Kim Ji-ae'), ('oName', 'trot')]
[('sName', 'Oöphoi'), ('oName', 'dark ambient')]
[('sName', 'Necrophagist'), ('oName', 'technical death metal')]
[('sName', 'Rivers of Nih

In [5]:
#P1811 list of episodes

queryString = """
SELECT DISTINCT ?s ?sName ?oName
WHERE {
   ?s wdt:P1811 ?o .
   ?s sc:name ?sName.
   ?o sc:name ?oName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q15594658'), ('sName', 'Cutthroat Kitchen'), ('oName', 'list of Cutthroat Kitchen episodes')]
[('s', 'http://www.wikidata.org/entity/Q15731300'), ('sName', 'Power Rangers Dino Charge'), ('oName', 'List of Power Rangers Dino Charge episodes')]
[('s', 'http://www.wikidata.org/entity/Q16039926'), ('sName', 'Garo: Makai no Hana'), ('oName', 'list of Garo: Makai no Hana episodes')]
[('s', 'http://www.wikidata.org/entity/Q13479326'), ('sName', 'RWBY'), ('oName', 'list of RWBY episodes')]
[('s', 'http://www.wikidata.org/entity/Q60740094'), ('sName', 'Danganronpa: The Animation'), ('oName', 'list of Danganronpa: The Animation episodes')]
[('s', 'http://www.wikidata.org/entity/Q2757796'), ('sName', 'Nisekoi'), ('oName', 'list of Nisekoi episodes')]
[('s', 'http://www.wikidata.org/entity/Q2555936'), ('sName', 'Meine Liebe'), ('oName', 'list of Meine Liebe episodes')]
[('s', 'http://www.wikidata.org/entity/Q15303732'), ('sName', 'The Tonight Show Sta

In [6]:
#Q15594658 Cutthroat Kitchen
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q15594658 ?p ?o .
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1811'), ('pName', 'list of episodes'), ('o', 'http://www.wikidata.org/entity/Q27665409'), ('oName', 'list of Cutthroat Kitchen episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('o', 'http://www.wikidata.org/entity/Q846662'), ('oName', 'game show')]
[('p', 'http://www.wikidata.org/prop/direct/P407'), ('pName', 'language of work or name'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q5398426'), ('oName', 'television series')]
[('p', 'http://www.wikidata.org/prop/direct/P437'), ('pName', 'distribution format'), ('o', 'http://www.wikidata.org/entity/Q723685'), ('oName', 'video on demand')]
[('p', 'htt

In [7]:
# Q5398426 television series
queryString = """
SELECT DISTINCT ?televisionSeries ?televisionSeriesName
WHERE {
    wd:Q15594658 wdt:P31 ?televisionSeries .
   ?televisionSeries sc:name ?televisionSeriesName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('televisionSeries', 'http://www.wikidata.org/entity/Q5398426'), ('televisionSeriesName', 'television series')]
1


Final query for this task

In [8]:
# write your final query
#Cutthroat Kitchen(wd:Q15594658) is an instance of (wdt:P31) of televisionSeries
queryString = """
SELECT DISTINCT ?televisionSeries ?televisionSeriesName
WHERE {
    wd:Q15594658 wdt:P31 ?televisionSeries .
   ?televisionSeries sc:name ?televisionSeriesName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('televisionSeries', 'http://www.wikidata.org/entity/Q5398426'), ('televisionSeriesName', 'television series')]
1


## Task 2
Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

In [24]:
# write your queries
queryString = """
SELECT DISTINCT ?tvseries ?tvseriesName
WHERE {
    ?tvseries wdt:P31 wd:Q5398426 .
   ?tvseries sc:name ?tvseriesName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('tvseries', 'http://www.wikidata.org/entity/Q1154217'), ('tvseriesName', 'Ironfist Chinmi')]
[('tvseries', 'http://www.wikidata.org/entity/Q691528'), ('tvseriesName', 'Wallace and Gromit')]
[('tvseries', 'http://www.wikidata.org/entity/Q8193809'), ('tvseriesName', 'Shomuni')]
[('tvseries', 'http://www.wikidata.org/entity/Q33323'), ('tvseriesName', 'Everybody Hates Chris')]
[('tvseries', 'http://www.wikidata.org/entity/Q500769'), ('tvseriesName', 'Freaks and Geeks')]
[('tvseries', 'http://www.wikidata.org/entity/Q8350955'), ('tvseriesName', 'Cordel Encantado')]
[('tvseries', 'http://www.wikidata.org/entity/Q1145054'), ('tvseriesName', 'The NBC Mystery Movie')]
[('tvseries', 'http://www.wikidata.org/entity/Q344081'), ('tvseriesName', 'Recess')]
[('tvseries', 'http://www.wikidata.org/entity/Q13647845'), ('tvseriesName', 'Grijpstra & De Gier')]
[('tvseries', 'http://www.wikidata.org/entity/Q708661'), ('tvseriesName', 'Lindenstraße')]
[('tvseries', 'http://www.wikidata.org/entity/

In [31]:
#Q27889400 Power Rangers Ninja Steel
queryString = """
SELECT DISTINCT ?pName ?oName
WHERE {
    wd:Q27889400 ?p ?o .
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pName', 'follows'), ('oName', 'Power Rangers Dino Charge')]
[('pName', 'original broadcaster'), ('oName', 'Nickelodeon')]
[('pName', 'distributed by'), ('oName', 'Saban Entertainment')]
[('pName', 'producer'), ('oName', 'Toei Company')]
[('pName', 'creator'), ('oName', 'Toei Company')]
[('pName', 'creator'), ('oName', 'Haim Saban')]
[('pName', 'executive producer'), ('oName', 'Haim Saban')]
[('pName', 'language of work or name'), ('oName', 'English')]
[('pName', 'country of origin'), ('oName', 'United States of America')]
[('pName', 'instance of'), ('oName', 'television series')]
[('pName', 'distribution format'), ('oName', 'video on demand')]
[('pName', 'distributed by'), ('oName', 'Netflix')]
[('pName', 'distributed by'), ('oName', 'MarVista Entertainment')]
[('pName', 'genre'), ('oName', 'tokusatsu')]
[('pName', 'producer'), ('oName', 'Saban Capital Group')]
[('pName', 'subclass of'), ('oName', 'Power Rangers')]
[('pName', 'has part'), ('oName', 'Power Rangers Super Ninja

In [34]:
#Q15737024 Power Rangers Ninja Steel
queryString = """
SELECT DISTINCT ?sName ?p ?pName
WHERE {
     ?s ?p wd:Q27889400 .
   ?p sc:name ?pName.
   ?s sc:name ?sName.

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('sName', 'Power Rangers Dino Charge'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('sName', 'Power Rangers Super Ninja Steel'), ('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series')]
[('sName', 'Power Rangers Ninja Steel, season 1'), ('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series')]
[('sName', 'Power Rangers Beast Morphers'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
4


In [130]:
#episodes per seasons 
# P179 part of the series
queryString = """
SELECT ?tvseriesName (COUNT(?seasonName) AS ?numseasons)
WHERE 
{
    ?tvseries wdt:P31 wd:Q5398426 ;
            sc:name ?tvseriesName .
    ?season wdt:P179 ?tvseries ;
       sc:name ?seasonName .

   FILTER REGEX(?seasonName, "season*")   
}
GROUP BY ?tvseriesName 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('tvseriesName', 'Fantasy Island'), ('numseasons', '7')]
[('tvseriesName', 'Mr. Novak'), ('numseasons', '2')]
[('tvseriesName', 'American Chopper: Senior vs. Junior'), ('numseasons', '4')]
[('tvseriesName', 'Family'), ('numseasons', '5')]
[('tvseriesName', 'Jem'), ('numseasons', '3')]
[('tvseriesName', 'Peter Punk'), ('numseasons', '2')]
[('tvseriesName', 'Danmarks Næste Topmodel'), ('numseasons', '6')]
[('tvseriesName', 'I liceali'), ('numseasons', '3')]
[('tvseriesName', 'The Eleventh Hour'), ('numseasons', '3')]
[('tvseriesName', 'Witchblade'), ('numseasons', '2')]
[('tvseriesName', 'Armchair Thriller'), ('numseasons', '2')]
[('tvseriesName', 'Survivor Philippines'), ('numseasons', '1')]
[('tvseriesName', 'Bob Hope Presents the Chrysler Theatre'), ('numseasons', '4')]
[('tvseriesName', 'The Colbys'), ('numseasons', '2')]
[('tvseriesName', 'Star Trek: Strange New Worlds'), ('numseasons', '1')]
[('tvseriesName', 'Class of the Titans'), ('numseasons', '1')]
[('tvseriesName', "

In [81]:
#Q1615887 Everybody Hates Chris, season 3
queryString = """
SELECT DISTINCT ?pName ?oName
WHERE {
    wd:Q1615887 ?p ?o .
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pName', 'part of the series'), ('oName', 'Everybody Hates Chris')]
[('pName', 'original language of film or TV show'), ('oName', 'English')]
[('pName', 'distribution format'), ('oName', 'video on demand')]
[('pName', 'instance of'), ('oName', 'television series season')]
[('pName', 'distribution format'), ('oName', 'digital download')]
[('pName', 'distributed by'), ('oName', 'iTunes')]
6


Final query for this task

In [97]:
# write your final query
queryString = """
SELECT ?season ?seasonName (COUNT(?seasonName) AS ?numseasons)
WHERE 
{
    ?tvseries wdt:P31 wd:Q5398426 ;
            sc:name ?tvseriesName .
    ?season wdt:P179 ?tvseries ;
       sc:name ?seasonName .

   FILTER REGEX(?seasonName, "season*")   
}
GROUP BY ?season ?seasonName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q64520767'), ('seasonName', 'Power Couple (Brazilian season 4)'), ('numseasons', '1')]
[('season', 'http://www.wikidata.org/entity/Q18819122'), ('seasonName', 'Banshee, season 3'), ('numseasons', '1')]
[('season', 'http://www.wikidata.org/entity/Q39072926'), ('seasonName', 'Instant Mom, season 2'), ('numseasons', '1')]
[('season', 'http://www.wikidata.org/entity/Q56229087'), ('seasonName', 'Megasztár, season 3'), ('numseasons', '1')]
[('season', 'http://www.wikidata.org/entity/Q66486674'), ('seasonName', 'Sacred Games, season 2'), ('numseasons', '1')]
[('season', 'http://www.wikidata.org/entity/Q60521583'), ('seasonName', 'Food Paradise, season 8'), ('numseasons', '1')]
[('season', 'http://www.wikidata.org/entity/Q60521582'), ('seasonName', 'Food Paradise, season 7'), ('numseasons', '1')]
[('season', 'http://www.wikidata.org/entity/Q3730143'), ('seasonName', 'The Big C, season 3'), ('numseasons', '1')]
[('season', 'http://www.wikidata

## Task 3
Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

In [100]:
# write your queries
#Q27889400 Power Rangers Ninja Steel
queryString = """
SELECT DISTINCT ?p ?pName ?oName
WHERE {
    wd:Q27889400 ?p ?o .
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('oName', 'Power Rangers Dino Charge')]
[('p', 'http://www.wikidata.org/prop/direct/P449'), ('pName', 'original broadcaster'), ('oName', 'Nickelodeon')]
[('p', 'http://www.wikidata.org/prop/direct/P750'), ('pName', 'distributed by'), ('oName', 'Saban Entertainment')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('pName', 'producer'), ('oName', 'Toei Company')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('pName', 'creator'), ('oName', 'Toei Company')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('pName', 'creator'), ('oName', 'Haim Saban')]
[('p', 'http://www.wikidata.org/prop/direct/P1431'), ('pName', 'executive producer'), ('oName', 'Haim Saban')]
[('p', 'http://www.wikidata.org/prop/direct/P407'), ('pName', 'language of work or name'), ('oName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin'), ('oName', 'United States of America')]
[

In [102]:
#P57 director
queryString = """
SELECT DISTINCT ?s ?sName ?oName
WHERE {
   ?s wdt:P57 ?o .
   ?s sc:name ?sName.
   ?o sc:name ?oName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8193809'), ('sName', 'Shomuni'), ('oName', 'Takayoshi Watanabe')]
[('s', 'http://www.wikidata.org/entity/Q21008514'), ('sName', 'Murder in Pacot'), ('oName', 'Raoul Peck')]
[('s', 'http://www.wikidata.org/entity/Q8452990'), ('sName', "Sunday League - Pepik Hnatek's Final Match"), ('oName', 'Jan Prušinovský')]
[('s', 'http://www.wikidata.org/entity/Q3611806'), ('sName', 'Alimony'), ('oName', 'Emmett J. Flynn')]
[('s', 'http://www.wikidata.org/entity/Q8350955'), ('sName', 'Cordel Encantado'), ('oName', 'Amora Mautner')]
[('s', 'http://www.wikidata.org/entity/Q20949971'), ('sName', 'Stolen Heaven'), ('oName', 'Andrew L. Stone')]
[('s', 'http://www.wikidata.org/entity/Q21646939'), ('sName', 'Roommate'), ('oName', 'Takeshi Furusawa')]
[('s', 'http://www.wikidata.org/entity/Q18040186'), ('sName', "Charlie Don't Surf"), ('oName', 'Jason Bloom')]
[('s', 'http://www.wikidata.org/entity/Q23419147'), ('sName', 'Anna Regina'), ('oName', 'Peter Kosmin

In [104]:
#Q21008514 Murder in Pacot
queryString = """
SELECT DISTINCT ?p ?pName ?oName
WHERE {
   wd:Q21008514 ?p ?o .
   ?p sc:name ?pName.
   ?o sc:name ?oName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pName', 'director'), ('oName', 'Raoul Peck')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('pName', 'producer'), ('oName', 'Rémi Grellety')]
[('p', 'http://www.wikidata.org/prop/direct/P344'), ('pName', 'director of photography'), ('oName', 'Éric Guichard')]
[('p', 'http://www.wikidata.org/prop/direct/P58'), ('pName', 'screenwriter'), ('oName', 'Lyonel Trouillot')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('oName', 'Thibault Vinçon')]
[('p', 'http://www.wikidata.org/prop/direct/P86'), ('pName', 'composer'), ('oName', 'Alexei Gennadjewitsch Aigi')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('oName', 'film')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin'), ('oName', 'France')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pName', 'original language of film or TV show'), ('oName', 'French')]
[('p', 'http://

In [121]:
#number of films actor/actress act
#P161 cast member
queryString = """
SELECT DISTINCT ?actor ?actorName (COUNT(?filmName) AS ?numfilm)
WHERE {
   ?film wdt:P161 ?actor .
   ?film sc:name ?filmName .
   ?actor sc:name ?actorName .
}
GROUP BY ?actor ?actorName
ORDER BY DESC (?numfilm)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q2735277'), ('actorName', 'Prem Nazir'), ('numfilm', '525')]
[('actor', 'http://www.wikidata.org/entity/Q946745'), ('actorName', 'Broncho Billy Anderson'), ('numfilm', '351')]
[('actor', 'http://www.wikidata.org/entity/Q379604'), ('actorName', 'Mithun Chakraborty'), ('numfilm', '346')]
[('actor', 'http://www.wikidata.org/entity/Q2721855'), ('actorName', 'Mammootty'), ('numfilm', '340')]
[('actor', 'http://www.wikidata.org/entity/Q72911'), ('actorName', 'Oliver Hardy'), ('numfilm', '335')]
[('actor', 'http://www.wikidata.org/entity/Q317574'), ('actorName', 'Mack Sennett'), ('numfilm', '297')]
[('actor', 'http://www.wikidata.org/entity/Q316955'), ('actorName', 'Michael Dorn'), ('numfilm', '294')]
[('actor', 'http://www.wikidata.org/entity/Q2050534'), ('actorName', 'Mohanlal'), ('numfilm', '291')]
[('actor', 'http://www.wikidata.org/entity/Q184572'), ('actorName', 'Martin Sheen'), ('numfilm', '286')]
[('actor', 'http://www.wikidata.org/en

In [131]:
#
queryString = """
SELECT DISTINCT ?tvseriesName ?actorName
WHERE {
    ?tvseries wdt:P31 wd:Q5398426 ;
            sc:name ?tvseriesName .
   ?tvseries wdt:P161 ?actor .
   ?actor sc:name ?actorName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('tvseriesName', 'Everybody Hates Chris'), ('actorName', 'Terry Crews')]
[('tvseriesName', 'Everybody Hates Chris'), ('actorName', 'Imani Hakim')]
[('tvseriesName', 'Everybody Hates Chris'), ('actorName', 'Tichina Arnold')]
[('tvseriesName', 'Everybody Hates Chris'), ('actorName', 'Tyler James Williams')]
[('tvseriesName', 'Everybody Hates Chris'), ('actorName', 'Vincent Martella')]
[('tvseriesName', 'Everybody Hates Chris'), ('actorName', 'Tequan Richmond')]
[('tvseriesName', 'Freaks and Geeks'), ('actorName', 'Linda Cardellini')]
[('tvseriesName', 'Freaks and Geeks'), ('actorName', 'Samm Levine')]
[('tvseriesName', 'Freaks and Geeks'), ('actorName', 'Jerry Messing')]
[('tvseriesName', 'Freaks and Geeks'), ('actorName', 'Busy Philipps')]
[('tvseriesName', 'Freaks and Geeks'), ('actorName', 'Becky Ann Baker')]
[('tvseriesName', 'Freaks and Geeks'), ('actorName', 'Thomas F. Wilson')]
[('tvseriesName', 'Freaks and Geeks'), ('actorName', 'Joe Flaherty')]
[('tvseriesName', 'Freaks

In [119]:
#Q8197668 FILM Idiot Love
queryString = """
SELECT DISTINCT ?pName ?oName
WHERE {
   wd:Q8197668 ?p ?o.
   ?o sc:name ?oName .
   ?p sc:name ?pName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pName', 'genre'), ('oName', 'drama')]
[('pName', 'cast member'), ('oName', 'Marc Cartes i Ivern')]
[('pName', 'cast member'), ('oName', 'Roger Casamajor')]
[('pName', 'cast member'), ('oName', 'Jordi Dauder')]
[('pName', 'cast member'), ('oName', 'Sophie Evans')]
[('pName', 'cast member'), ('oName', 'Santi Millán')]
[('pName', 'cast member'), ('oName', 'Mercè Pons')]
[('pName', 'cast member'), ('oName', 'Cayetana Guillén Cuervo')]
[('pName', 'instance of'), ('oName', 'film')]
[('pName', 'original language of film or TV show'), ('oName', 'Spanish')]
[('pName', 'country of origin'), ('oName', 'Spain')]
[('pName', 'director'), ('oName', 'Ventura Pons')]
[('pName', 'composer'), ('oName', 'Carles Cases')]
13


In [122]:
# Q2735277 actor Prem Nazir
queryString = """
SELECT DISTINCT ?pName ?oName
WHERE {
   wd:Q2735277 ?p ?o.
   ?o sc:name ?oName .
   ?p sc:name ?pName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pName', 'native language'), ('oName', 'Malayalam')]
[('pName', 'occupation'), ('oName', 'actor')]
[('pName', 'filmography'), ('oName', 'Prem Nazir filmography')]
[('pName', 'religion'), ('oName', 'Islam')]
[('pName', 'languages spoken, written or signed'), ('oName', 'Malayalam')]
[('pName', 'award received'), ('oName', 'Padma Shri in arts')]
[('pName', 'award received'), ('oName', 'Padma Bhushan')]
[('pName', 'place of birth'), ('oName', 'Chirayinkizhu Block Panchayat')]
[('pName', 'place of death'), ('oName', 'Chennai')]
[('pName', 'sex or gender'), ('oName', 'male')]
[('pName', 'country of citizenship'), ('oName', 'British India')]
[('pName', 'country of citizenship'), ('oName', 'India')]
[('pName', 'country of citizenship'), ('oName', 'Dominion of India')]
[('pName', 'instance of'), ('oName', 'human')]
[('pName', 'sibling'), ('oName', 'Prem Nawas')]
[('pName', 'child'), ('oName', 'Shanawas')]
[('pName', 'educated at'), ('oName', 'St. Berchmans College')]
[('pName', 'given

Final query for this task

In [135]:
# write your final query
queryString = """
SELECT DISTINCT ?actor ?actorName (COUNT(?tvseriesName) AS ?numseries)
WHERE {
    ?tvseries wdt:P31 wd:Q5398426 ;
            sc:name ?tvseriesName .
   ?tvseries wdt:P161 ?actor .
   ?actor sc:name ?actorName .
}
GROUP BY ?actor ?actorName
ORDER BY DESC (?numseries)

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q12042457'), ('actorName', 'Oldřich Vlach'), ('numseries', '26')]
[('actor', 'http://www.wikidata.org/entity/Q10292775'), ('actorName', 'Guilherme Filipe'), ('numseries', '21')]
[('actor', 'http://www.wikidata.org/entity/Q2058853'), ('actorName', 'Hernesta Derbez'), ('numseries', '21')]
[('actor', 'http://www.wikidata.org/entity/Q741577'), ('actorName', 'Radoslav Brzobohatý'), ('numseries', '20')]
[('actor', 'http://www.wikidata.org/entity/Q1683633'), ('actorName', 'Jaroslav Moučka'), ('numseries', '20')]
[('actor', 'http://www.wikidata.org/entity/Q101209973'), ('actorName', 'Jiří Wohanka'), ('numseries', '20')]
[('actor', 'http://www.wikidata.org/entity/Q11752586'), ('actorName', 'Ladislav Trojan'), ('numseries', '20')]
[('actor', 'http://www.wikidata.org/entity/Q374447'), ('actorName', 'Sam Anderson'), ('numseries', '20')]
[('actor', 'http://www.wikidata.org/entity/Q29525305'), ('actorName', 'Zdeněk Žák'), ('numseries', '19')]
[('act

## Task 4
Check who is the actor who acted in more films while working on "How I met your mother" (the result set must be a list of triples actor/actress IRI, label and #films).

In [150]:
# write your queries
#actor/actress act in "How I met your mother"
#P161 cast member
queryString = """
SELECT DISTINCT ?film ?filmName ?actorName
WHERE {
   ?film wdt:P161 ?actor .
   ?film sc:name ?filmName .
   ?actor sc:name ?actorName .
   FILTER REGEX(?filmName , "[Mm]other*")
   FILTER REGEX(?filmName , "[Hh]ow*")
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q147235'), ('filmName', 'How I Met Your Mother'), ('actorName', 'Annie Ilonzeh')]
[('film', 'http://www.wikidata.org/entity/Q147235'), ('filmName', 'How I Met Your Mother'), ('actorName', 'Julianna Guill')]
[('film', 'http://www.wikidata.org/entity/Q147235'), ('filmName', 'How I Met Your Mother'), ('actorName', 'Benjamin Koldyke')]
[('film', 'http://www.wikidata.org/entity/Q147235'), ('filmName', 'How I Met Your Mother'), ('actorName', 'Nikki Griffin')]
[('film', 'http://www.wikidata.org/entity/Q147235'), ('filmName', 'How I Met Your Mother'), ('actorName', 'John Cho')]
[('film', 'http://www.wikidata.org/entity/Q147235'), ('filmName', 'How I Met Your Mother'), ('actorName', 'Mircea Monroe')]
[('film', 'http://www.wikidata.org/entity/Q147235'), ('filmName', 'How I Met Your Mother'), ('actorName', 'Virginia Williams')]
[('film', 'http://www.wikidata.org/entity/Q147235'), ('filmName', 'How I Met Your Mother'), ('actorName', 'Kate Micucci')

In [ ]:
queryString = """
SELECT ?season ?seasonName (COUNT(?seasonName) AS ?numseasons)
WHERE 
{
    ?tvseries wdt:P31 wd:Q5398426 ;
            sc:name ?tvseriesName .
    ?season wdt:P179 ?tvseries ;
       sc:name ?seasonName .

   FILTER REGEX(?seasonName, "season*")   
}
GROUP BY ?season ?seasonName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results


In [139]:
# Q147235 How I Met Your Mother
queryString = """
SELECT DISTINCT ?p ?pName #?oName
WHERE {
   wd:Q147235 ?p ?o .
   ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P449'), ('pName', 'original broadcaster')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P86'), ('pName', 'composer')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pName', 'original language of film or TV show')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P840'), ('pName', 'narrative location')]
[('p', 'http://www.wikidata.org/prop/direct/P437'), ('pName', 'distribution format')]
[('p', 'http://www.wikidata.org/prop/direct/P750'), ('pName', 'distributed 

In [147]:
# Q147235 How I Met Your Mother
queryString = """
SELECT DISTINCT ?actor ?actorName (COUNT(?film) AS ?numfilm)
WHERE {
   wd:Q147235 wdt:P161 ?actor .
   ?actor sc:name ?actorName .
   ?film wdt:P161 ?actor .
}
GROUP BY ?actor ?actorName
ORDER BY DESC (?numfilm)

LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q485310'), ('actorName', 'Neil Patrick Harris'), ('numfilm', '187')]
[('actor', 'http://www.wikidata.org/entity/Q200566'), ('actorName', 'Cobie Smulders'), ('numfilm', '173')]
[('actor', 'http://www.wikidata.org/entity/Q199927'), ('actorName', 'Alyson Hannigan'), ('numfilm', '170')]
[('actor', 'http://www.wikidata.org/entity/Q202304'), ('actorName', 'Jason Segel'), ('numfilm', '170')]
[('actor', 'http://www.wikidata.org/entity/Q223455'), ('actorName', 'Josh Radnor'), ('numfilm', '155')]
[('actor', 'http://www.wikidata.org/entity/Q192165'), ('actorName', 'Danny Glover'), ('numfilm', '117')]
[('actor', 'http://www.wikidata.org/entity/Q471018'), ('actorName', 'Ernie Hudson'), ('numfilm', '75')]
[('actor', 'http://www.wikidata.org/entity/Q536437'), ('actorName', 'Edward Herrmann'), ('numfilm', '74')]
[('actor', 'http://www.wikidata.org/entity/Q311271'), ('actorName', 'John Lithgow'), ('numfilm', '74')]
[('actor', 'http://www.wikidata.org/e

Final query for this task

In [148]:
# write your final query
queryString = """
SELECT DISTINCT ?actor ?actorName (COUNT(?film) AS ?numfilm)
WHERE {
   wd:Q147235 wdt:P161 ?actor .
   ?actor sc:name ?actorName .
   ?film wdt:P161 ?actor .
}
GROUP BY ?actor ?actorName
ORDER BY DESC (?numfilm)

LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q485310'), ('actorName', 'Neil Patrick Harris'), ('numfilm', '187')]
[('actor', 'http://www.wikidata.org/entity/Q200566'), ('actorName', 'Cobie Smulders'), ('numfilm', '173')]
[('actor', 'http://www.wikidata.org/entity/Q199927'), ('actorName', 'Alyson Hannigan'), ('numfilm', '170')]
[('actor', 'http://www.wikidata.org/entity/Q202304'), ('actorName', 'Jason Segel'), ('numfilm', '170')]
[('actor', 'http://www.wikidata.org/entity/Q223455'), ('actorName', 'Josh Radnor'), ('numfilm', '155')]
[('actor', 'http://www.wikidata.org/entity/Q192165'), ('actorName', 'Danny Glover'), ('numfilm', '117')]
[('actor', 'http://www.wikidata.org/entity/Q471018'), ('actorName', 'Ernie Hudson'), ('numfilm', '75')]
[('actor', 'http://www.wikidata.org/entity/Q536437'), ('actorName', 'Edward Herrmann'), ('numfilm', '74')]
[('actor', 'http://www.wikidata.org/entity/Q311271'), ('actorName', 'John Lithgow'), ('numfilm', '74')]
[('actor', 'http://www.wikidata.org/e

## Task 5
Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

In [149]:
# write your queries
# Q147235 How I Met Your Mother number of films
# ?tvseries wdt:P31 wd:Q5398426 . instance of tvseries
queryString = """
SELECT DISTINCT 
WHERE {
   wd:Q147235 wdt:P161 ?actor .
   ?actor sc:name ?actorName .
   ?film wdt:P161 ?actor .
}
GROUP BY ?actor ?actorName
ORDER BY DESC (?numfilm)

LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q485310'), ('actorName', 'Neil Patrick Harris'), ('numfilm', '187')]
[('actor', 'http://www.wikidata.org/entity/Q200566'), ('actorName', 'Cobie Smulders'), ('numfilm', '173')]
[('actor', 'http://www.wikidata.org/entity/Q199927'), ('actorName', 'Alyson Hannigan'), ('numfilm', '170')]
[('actor', 'http://www.wikidata.org/entity/Q202304'), ('actorName', 'Jason Segel'), ('numfilm', '170')]
[('actor', 'http://www.wikidata.org/entity/Q223455'), ('actorName', 'Josh Radnor'), ('numfilm', '155')]
[('actor', 'http://www.wikidata.org/entity/Q192165'), ('actorName', 'Danny Glover'), ('numfilm', '117')]
[('actor', 'http://www.wikidata.org/entity/Q471018'), ('actorName', 'Ernie Hudson'), ('numfilm', '75')]
[('actor', 'http://www.wikidata.org/entity/Q536437'), ('actorName', 'Edward Herrmann'), ('numfilm', '74')]
[('actor', 'http://www.wikidata.org/entity/Q311271'), ('actorName', 'John Lithgow'), ('numfilm', '74')]
[('actor', 'http://www.wikidata.org/e

In [155]:
#Q5398426 tvseries
queryString = """
SELECT DISTINCT ?tvseries ?tvseriesName
WHERE {
   ?tvseries wdt:P31 wd:Q5398426 .
   ?tvseries sc:name ?tvseriesName .
   FILTER REGEX(?tvseriesName , "[Oo]ffice*")
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('tvseries', 'http://www.wikidata.org/entity/Q28419054'), ('tvseriesName', 'Radiant Office')]
[('tvseries', 'http://www.wikidata.org/entity/Q23829'), ('tvseriesName', 'The Office')]
[('tvseries', 'http://www.wikidata.org/entity/Q23831'), ('tvseriesName', 'The Office')]
[('tvseries', 'http://www.wikidata.org/entity/Q6532522'), ('tvseriesName', "Let's Play Post Office")]
[('tvseries', 'http://www.wikidata.org/entity/Q7079129'), ('tvseriesName', 'Office Office')]
[('tvseries', 'http://www.wikidata.org/entity/Q65043588'), ('tvseriesName', 'The Office')]
[('tvseries', 'http://www.wikidata.org/entity/Q5759839'), ('tvseriesName', 'Office Girls')]
[('tvseries', 'http://www.wikidata.org/entity/Q6928472'), ('tvseriesName', 'Mr. Box Office')]
[('tvseries', 'http://www.wikidata.org/entity/Q6402787'), ('tvseriesName', 'Queen of the Office')]
[('tvseries', 'http://www.wikidata.org/entity/Q7764423'), ('tvseriesName', 'The Singing Office')]
10


In [160]:
#P495 country of origin
queryString = """
SELECT DISTINCT ?p1 ?p1Name ?o1Name #?p2Name ?o2Name
WHERE {
   wd:Q23829 ?p1 ?o1 .
   wd:Q23831 ?p2 ?o2 .
   ?p1 sc:name ?p1Name .
   ?o1 sc:name ?o1Name .
    ?p2 sc:name ?p2Name .
   ?o2 sc:name ?o2Name .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p1', 'http://www.wikidata.org/prop/direct/P136'), ('p1Name', 'genre'), ('o1Name', 'sitcom')]
[('p1', 'http://www.wikidata.org/prop/direct/P136'), ('p1Name', 'genre'), ('o1Name', 'mockumentary')]
[('p1', 'http://www.wikidata.org/prop/direct/P1552'), ('p1Name', 'has quality'), ('o1Name', 'breaking the fourth wall')]
[('p1', 'http://www.wikidata.org/prop/direct/P161'), ('p1Name', 'cast member'), ('o1Name', 'Patrick Baladi')]
[('p1', 'http://www.wikidata.org/prop/direct/P161'), ('p1Name', 'cast member'), ('o1Name', 'Ralph Ineson')]
[('p1', 'http://www.wikidata.org/prop/direct/P161'), ('p1Name', 'cast member'), ('o1Name', 'Ricky Gervais')]
[('p1', 'http://www.wikidata.org/prop/direct/P161'), ('p1Name', 'cast member'), ('o1Name', 'Martin Freeman')]
[('p1', 'http://www.wikidata.org/prop/direct/P161'), ('p1Name', 'cast member'), ('o1Name', 'Lucy Davis')]
[('p1', 'http://www.wikidata.org/prop/direct/P161'), ('p1Name', 'cast member'), ('o1Name', 'Oliver Chris')]
[('p1', 'http://www.wi

In [162]:
#P495 country of origin
queryString = """
SELECT DISTINCT ?country1 ?country1Name ?country2 ?country2Name
WHERE {
   wd:Q23829 wdt:P495 ?country1 .
   wd:Q23831 wdt:P495 ?country2 .
   ?country1 sc:name ?country1Name .
   ?country2 sc:name ?country2Name .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country1', 'http://www.wikidata.org/entity/Q145'), ('country1Name', 'United Kingdom'), ('country2', 'http://www.wikidata.org/entity/Q30'), ('country2Name', 'United States of America')]
1


In [214]:
queryString = """
SELECT DISTINCT wd:Q23831 ?theofficeName ?numseaonOffice ?numepisodeoffice ?numofficeactor
WHERE
{              
    {
        SELECT DISTINCT ?theofficeName (COUNT(?officeseasonName) AS ?numseaonOffice)  
        WHERE
        {
            wd:Q23831 sc:name ?theofficeName .
            ?officeseason wdt:P179 wd:Q23831 ;
               sc:name ?officeseasonName .
           FILTER REGEX(?officeseasonName, "season*") 
        }
       GROUP BY ?theofficeName
    }

    {
        SELECT DISTINCT ?theofficeName (COUNT(?officeepisodeName) AS ?numepisodeoffice)  
        WHERE
        {
            wd:Q23831 sc:name ?theofficeName .
            ?officeepisode wdt:P179 wd:Q23831 ;
               sc:name ?officeepisodeName .
        }
       GROUP BY ?theofficeName
   }


   {
       SELECT DISTINCT ?theofficeName (COUNT(?officeactorName) AS ?numofficeactor)  
        WHERE
        {
            wd:Q23831 sc:name ?theofficeName .
            wd:Q23831 wdt:P161 ?officeactor .
            ?officeactor sc:name ?officeactorName .

        }
       GROUP BY ?theofficeName
   }
}
"""

print("Results")
x=run_query(queryString)

Results
[('callret-0', 'http://www.wikidata.org/entity/Q23831'), ('theofficeName', 'The Office'), ('numseaonOffice', '9'), ('numepisodeoffice', '226'), ('numofficeactor', '25')]
1


In [210]:
queryString = """
SELECT DISTINCT wd:Q147235 ?himymName ?numseaonHimym ?numepisodeHimym ?numhimymactor
WHERE
{ 
    {
        SELECT DISTINCT ?himymName (COUNT(?himymseasonName) AS ?numseaonHimym)  
        WHERE
        {
            wd:Q147235 sc:name ?himymName .
            ?himymseason wdt:P179  wd:Q23831 ; 
                   sc:name ?himymseasonName .
            FILTER REGEX(?himymseasonName, "season*") 
        }
       GROUP BY ?himymName
   }
    {
        SELECT DISTINCT ?himymName (COUNT(?himymepisodeName) AS ?numepisodeHimym)  
        WHERE
        {
            wd:Q147235 sc:name ?himymName .
            ?himymepisode wdt:P179  wd:Q23831 ; 
                   sc:name ?himymepisodeName .
        }
       GROUP BY ?himymName
   }


   {
       SELECT DISTINCT ?himymName (COUNT(?himymactorName) AS ?numhimymactor)  
        WHERE
        {
            wd:Q147235 sc:name ?himymName .
            wd:Q147235 wdt:P161 ?himymactor .
            ?himymactor sc:name ?himymactorName .

        }
       GROUP BY ?himymName
   }

}

"""
print("Results")
x=run_query(queryString)

Results
[('callret-0', 'http://www.wikidata.org/entity/Q147235'), ('himymName', 'How I Met Your Mother'), ('numseaonHimym', '9'), ('numepisodeHimym', '226'), ('numhimymactor', '480')]
1


In [215]:
#compare number of seasons
# wd:Q23831 the tv series "The Office (US)"
# wd:Q147235 How I Met Your Mother (HIMYM)
queryString = """
SELECT DISTINCT 
wd:Q23831 ?theofficeName ?numseaonOffice ?numepisodeoffice ?numofficeactor
wd:Q147235 ?himymName ?numseaonHimym ?numepisodeHimym ?numhimymactor
WHERE 
{
        {              
            {
                SELECT DISTINCT ?theofficeName (COUNT(?officeseasonName) AS ?numseaonOffice)  
                WHERE
                {
                    wd:Q23831 sc:name ?theofficeName .
                    ?officeseason wdt:P179 wd:Q23831 ;
                       sc:name ?officeseasonName .
                   FILTER REGEX(?officeseasonName, "season*") 
                }
               GROUP BY ?theofficeName
            }

            {
                SELECT DISTINCT ?theofficeName (COUNT(?officeepisodeName) AS ?numepisodeoffice)  
                WHERE
                {
                    wd:Q23831 sc:name ?theofficeName .
                    ?officeepisode wdt:P179 wd:Q23831 ;
                       sc:name ?officeepisodeName .
                }
               GROUP BY ?theofficeName
           }


           {
               SELECT DISTINCT ?theofficeName (COUNT(?officeactorName) AS ?numofficeactor)  
                WHERE
                {
                    wd:Q23831 sc:name ?theofficeName .
                    wd:Q23831 wdt:P161 ?officeactor .
                    ?officeactor sc:name ?officeactorName .

                }
               GROUP BY ?theofficeName
           }
        }

        { 
            {
                SELECT DISTINCT ?himymName (COUNT(?himymseasonName) AS ?numseaonHimym)  
                WHERE
                {
                    wd:Q147235 sc:name ?himymName .
                    ?himymseason wdt:P179  wd:Q23831 ; 
                           sc:name ?himymseasonName .
                    FILTER REGEX(?himymseasonName, "season*") 
                }
               GROUP BY ?himymName
           }
            {
                SELECT DISTINCT ?himymName (COUNT(?himymepisodeName) AS ?numepisodeHimym)  
                WHERE
                {
                    wd:Q147235 sc:name ?himymName .
                    ?himymepisode wdt:P179  wd:Q23831 ; 
                           sc:name ?himymepisodeName .
                }
               GROUP BY ?himymName
           }


           {
               SELECT DISTINCT ?himymName (COUNT(?himymactorName) AS ?numhimymactor)  
                WHERE
                {
                    wd:Q147235 sc:name ?himymName .
                    wd:Q147235 wdt:P161 ?himymactor .
                    ?himymactor sc:name ?himymactorName .

                }
               GROUP BY ?himymName
           }

        }


}


LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('callret-0', 'http://www.wikidata.org/entity/Q23831'), ('theofficeName', 'The Office'), ('numseaonOffice', '9'), ('numepisodeoffice', '226'), ('numofficeactor', '25'), ('callret-5', 'http://www.wikidata.org/entity/Q147235'), ('himymName', 'How I Met Your Mother'), ('numseaonHimym', '9'), ('numepisodeHimym', '226'), ('numhimymactor', '480')]
1


Final query for this task

In [216]:
# write your final query
queryString = """
SELECT DISTINCT 
wd:Q23831 ?theofficeName ?numseaonOffice ?numepisodeoffice ?numofficeactor
wd:Q147235 ?himymName ?numseaonHimym ?numepisodeHimym ?numhimymactor
WHERE 
{
        {              
            {
                SELECT DISTINCT ?theofficeName (COUNT(?officeseasonName) AS ?numseaonOffice)  
                WHERE
                {
                    wd:Q23831 sc:name ?theofficeName .
                    ?officeseason wdt:P179 wd:Q23831 ;
                       sc:name ?officeseasonName .
                   FILTER REGEX(?officeseasonName, "season*") 
                }
               GROUP BY ?theofficeName
            }

            {
                SELECT DISTINCT ?theofficeName (COUNT(?officeepisodeName) AS ?numepisodeoffice)  
                WHERE
                {
                    wd:Q23831 sc:name ?theofficeName .
                    ?officeepisode wdt:P179 wd:Q23831 ;
                       sc:name ?officeepisodeName .
                }
               GROUP BY ?theofficeName
           }


           {
               SELECT DISTINCT ?theofficeName (COUNT(?officeactorName) AS ?numofficeactor)  
                WHERE
                {
                    wd:Q23831 sc:name ?theofficeName .
                    wd:Q23831 wdt:P161 ?officeactor .
                    ?officeactor sc:name ?officeactorName .

                }
               GROUP BY ?theofficeName
           }
        }

        { 
            {
                SELECT DISTINCT ?himymName (COUNT(?himymseasonName) AS ?numseaonHimym)  
                WHERE
                {
                    wd:Q147235 sc:name ?himymName .
                    ?himymseason wdt:P179  wd:Q23831 ; 
                           sc:name ?himymseasonName .
                    FILTER REGEX(?himymseasonName, "season*") 
                }
               GROUP BY ?himymName
           }
            {
                SELECT DISTINCT ?himymName (COUNT(?himymepisodeName) AS ?numepisodeHimym)  
                WHERE
                {
                    wd:Q147235 sc:name ?himymName .
                    ?himymepisode wdt:P179  wd:Q23831 ; 
                           sc:name ?himymepisodeName .
                }
               GROUP BY ?himymName
           }


           {
               SELECT DISTINCT ?himymName (COUNT(?himymactorName) AS ?numhimymactor)  
                WHERE
                {
                    wd:Q147235 sc:name ?himymName .
                    wd:Q147235 wdt:P161 ?himymactor .
                    ?himymactor sc:name ?himymactorName .

                }
               GROUP BY ?himymName
           }

        }


}


LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('callret-0', 'http://www.wikidata.org/entity/Q23831'), ('theofficeName', 'The Office'), ('numseaonOffice', '9'), ('numepisodeoffice', '226'), ('numofficeactor', '25'), ('callret-5', 'http://www.wikidata.org/entity/Q147235'), ('himymName', 'How I Met Your Mother'), ('numseaonHimym', '9'), ('numepisodeHimym', '226'), ('numhimymactor', '480')]
1


## Task 6
Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

In [224]:
# write your queries
queryString = """
SELECT DISTINCT ?actor ?actorName
WHERE {
    ?tvseries wdt:P31 wd:Q5398426 ;
            sc:name ?tvseriesName.
    ?tvseries wdt:P161 ?actor .
    ?actor sc:name ?actorName .
    FILTER REGEX(?actorName, "Kevin Bacon")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('actorName', 'Kevin Bacon')]
1


In [239]:
#Q3454165 Kevin Bacon
queryString = """
SELECT DISTINCT  (COUNT(?actorBaconnum2Name) AS ?howmany)
WHERE 
{
    ?tvseries1 wdt:P161 wd:Q3454165 .
    ?tvseries2 wdt:P161 wd:Q3454165 .
    ?tvseries1 sc:name ?tvseries1Name .
    ?tvseries1 wdt:P161 ?actorBaconnum1 .
    ?tvseries2 wdt:P161 ?actorBaconnum1 .
    ?tvseries2 wdt:P161 ?actorBaconnum2 .
    ?tvseries1 sc:name ?tvseries1Name .
    ?tvseries2 sc:name ?tvseries2Name .
    ?actorBaconnum1 sc:name ?actorBaconnum1Name .
    ?actorBaconnum2 sc:name ?actorBaconnum2Name .
    FILTER (?tvseries1 != ?tvseries2)
    FILTER (?actorBaconnum1Name != ?actorBaconnum2Name)

} 

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('howmany', '67568')]
1


Final query for this task

In [240]:
# write your final query
queryString = """
SELECT DISTINCT  (COUNT(?actorBaconnum2Name) AS ?howmany)
WHERE 
{
    ?tvseries1 wdt:P161 wd:Q3454165 .
    ?tvseries2 wdt:P161 wd:Q3454165 .
    ?tvseries1 sc:name ?tvseries1Name .
    ?tvseries1 wdt:P161 ?actorBaconnum1 .
    ?tvseries2 wdt:P161 ?actorBaconnum1 .
    ?tvseries2 wdt:P161 ?actorBaconnum2 .
    ?tvseries1 sc:name ?tvseries1Name .
    ?tvseries2 sc:name ?tvseries2Name .
    ?actorBaconnum1 sc:name ?actorBaconnum1Name .
    ?actorBaconnum2 sc:name ?actorBaconnum2Name .
    FILTER (?tvseries1 != ?tvseries2)
    FILTER (?actorBaconnum1Name != ?actorBaconnum2Name)

} 

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('howmany', '67568')]
1


## Task 7
Consider the actors who are members of the cast of HIMYM. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

In [244]:
# write your queries
queryString = """
SELECT DISTINCT ?tvseries ?tvseriesName
WHERE 
{
    wd:Q147235 wdt:P161 ?actor .
    ?actor sc:name ?actorName .
    ?tvseries wdt:P161 ?actor .
    ?tvseries sc:name ?tvseriesName .
} 

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('tvseries', 'http://www.wikidata.org/entity/Q500769'), ('tvseriesName', 'Freaks and Geeks')]
[('tvseries', 'http://www.wikidata.org/entity/Q17049828'), ('tvseriesName', 'Bad Samaritans')]
[('tvseries', 'http://www.wikidata.org/entity/Q377923'), ('tvseriesName', 'Some Came Running')]
[('tvseries', 'http://www.wikidata.org/entity/Q15072819'), ('tvseriesName', 'Halt and Catch Fire')]
[('tvseries', 'http://www.wikidata.org/entity/Q17627616'), ('tvseriesName', 'Miss Meadows')]
[('tvseries', 'http://www.wikidata.org/entity/Q17635617'), ('tvseriesName', 'Nantucket')]
[('tvseries', 'http://www.wikidata.org/entity/Q1197932'), ('tvseriesName', 'Magic Mike')]
[('tvseries', 'http://www.wikidata.org/entity/Q18151386'), ('tvseriesName', 'Kirby Buckets')]
[('tvseries', 'http://www.wikidata.org/entity/Q22908690'), ('tvseriesName', 'The Good Place')]
[('tvseries', 'http://www.wikidata.org/entity/Q210676'), ('tvseriesName', 'Nashville')]
[('tvseries', 'http://www.wikidata.org/entity/Q1607657')

In [259]:
queryString = """
SELECT DISTINCT ?awardName
WHERE 
{
    wd:Q147235 wdt:P166 ?award .
    ?award sc:name ?awardName .
} 
GROUP BY ?tvseriesName

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('awardName', 'Primetime Emmy Award for Outstanding Art Direction for a Contemporary Program')]
[('awardName', "Critics' Choice Television Award for Best Supporting Actor in a Comedy Series")]
[('awardName', 'International TV Audience Award for Best Comedy TV Series')]
[('awardName', 'Primetime Emmy Award for Outstanding Cinematography for a Multi-Camera Series')]
[('awardName', 'Primetime Emmy Award for Outstanding Multi-Camera Picture Editing for a Comedy Series')]
[('awardName', 'Primetime Emmy Award for Outstanding Single-Camera Picture Editing for a Comedy Series')]
[('awardName', 'Streamy Awards')]
7


In [301]:
#wd:Q147235 HIMYM 
#P166 award received
queryString = """
SELECT DISTINCT ?tvseries ?tvseriesName (COUNT(?awardName) AS ?numaward)
WHERE 
{
    wd:Q147235 wdt:P161 ?actor .
    ?actor sc:name ?actorName .
    
    ?tvseries wdt:P161 ?actor ;
            wdt:P31 wd:Q5398426 ;
            sc:name ?tvseriesName ;
            wdt:P166 ?award .
    ?award sc:name ?awardName .
    
} 
GROUP BY ?tvseries ?tvseriesName ?actorName
ORDER BY DESC (?numaward)

LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('tvseries', 'http://www.wikidata.org/entity/Q23628'), ('tvseriesName', 'The Sopranos'), ('numaward', '22')]
[('tvseries', 'http://www.wikidata.org/entity/Q1079'), ('tvseriesName', 'Breaking Bad'), ('numaward', '21')]
[('tvseries', 'http://www.wikidata.org/entity/Q1132439'), ('tvseriesName', 'The Practice'), ('numaward', '12')]
[('tvseries', 'http://www.wikidata.org/entity/Q16756'), ('tvseriesName', 'Modern Family'), ('numaward', '11')]
[('tvseries', 'http://www.wikidata.org/entity/Q8539'), ('tvseriesName', 'The Big Bang Theory'), ('numaward', '10')]
[('tvseries', 'http://www.wikidata.org/entity/Q79784'), ('tvseriesName', 'Friends'), ('numaward', '10')]
[('tvseries', 'http://www.wikidata.org/entity/Q1136370'), ('tvseriesName', 'General Hospital'), ('numaward', '10')]
[('tvseries', 'http://www.wikidata.org/entity/Q244803'), ('tvseriesName', 'Ally McBeal'), ('numaward', '9')]
[('tvseries', 'http://www.wikidata.org/entity/Q438406'), ('tvseriesName', "Grey's Anatomy"), ('numaward'

Final query for this task

In [310]:
# write your final query
queryString = """
SELECT DISTINCT ?tvseries ?tvseriesName (COUNT(?awardName) AS ?numaward)
WHERE 
{
    wd:Q147235 wdt:P161 ?actor .
    ?actor sc:name ?actorName .
    
    ?tvseries wdt:P161 ?actor ;
            wdt:P31 wd:Q5398426 ;
            sc:name ?tvseriesName ;
            wdt:P166 ?award .
    ?award sc:name ?awardName .
} 
GROUP BY ?tvseries ?tvseriesName ?actorName
HAVING (COUNT(?awardName) > 2)
ORDER BY DESC (?numaward)


LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('tvseries', 'http://www.wikidata.org/entity/Q23628'), ('tvseriesName', 'The Sopranos'), ('numaward', '22')]
[('tvseries', 'http://www.wikidata.org/entity/Q1079'), ('tvseriesName', 'Breaking Bad'), ('numaward', '21')]
[('tvseries', 'http://www.wikidata.org/entity/Q1132439'), ('tvseriesName', 'The Practice'), ('numaward', '12')]
[('tvseries', 'http://www.wikidata.org/entity/Q16756'), ('tvseriesName', 'Modern Family'), ('numaward', '11')]
[('tvseries', 'http://www.wikidata.org/entity/Q8539'), ('tvseriesName', 'The Big Bang Theory'), ('numaward', '10')]
[('tvseries', 'http://www.wikidata.org/entity/Q79784'), ('tvseriesName', 'Friends'), ('numaward', '10')]
[('tvseries', 'http://www.wikidata.org/entity/Q1136370'), ('tvseriesName', 'General Hospital'), ('numaward', '10')]
[('tvseries', 'http://www.wikidata.org/entity/Q244803'), ('tvseriesName', 'Ally McBeal'), ('numaward', '9')]
[('tvseries', 'http://www.wikidata.org/entity/Q438406'), ('tvseriesName', "Grey's Anatomy"), ('numaward'